In [1]:
import jax.numpy as np
import numpy as onp
from functools import partial
from jax import vmap
from jax.lax import scan
from jax.lax import cond
from jax import random
from jax import jit
from jax import jacrev
from jax.lax import stop_gradient
import matplotlib.pylab as plt
import matplotlib as mpl
from sklearn import manifold, datasets
import seaborn as sns
import jax

from jax.config import config
config.update("jax_debug_nans", True)

import jax.numpy as np
from utils import MidpointNormalize, load_data
from jax import random, flatten_util, vjp, jvp, custom_vjp, jacfwd, jacrev, vmap, grad
import matplotlib.pylab as plt
import seaborn as sns
from tsne_jax import *
import time

In [6]:
from sklearn.datasets import make_blobs, make_classification
#X, y = load_data(40)
X, y = make_blobs(n_samples=50, n_features=1000, centers=4, random_state=0, shuffle=False, cluster_std=[0.1, 3, 3, 3])
#X, y = make_classification(n_classes=5, n_samples=50, n_features=1000, random_state=42)
key = random.PRNGKey(41)
#X = onp.array(random.normal(key, shape=(50, 50)))
y_guess = random.normal(key, shape=(X.shape[0], 2))
#Y_star = TSNE(n_components=2, learning_rate=200, init=onp.array(y_guess), perplexity=30).fit_transform(X)
Y_star = tsne_fwd(X, y_guess)

X_flat, X_unflattener = flatten_util.ravel_pytree(X)   # row-wise
Y_flat, Y_unflattener = flatten_util.ravel_pytree(Y_star) 

===> Finding 49 nearest neighbors using Annoy approximate search using euclidean distance...
   --> Time elapsed: 0.01 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.00 seconds
===> Running optimization with exaggeration=12.00, lr=200.00 for 250 iterations...
Iteration   50, KL divergence 0.8921, 50 iterations in 0.2886 sec
Iteration  100, KL divergence 0.9531, 50 iterations in 0.2763 sec
Iteration  150, KL divergence 0.9984, 50 iterations in 0.2764 sec
Iteration  200, KL divergence 0.8934, 50 iterations in 0.2850 sec
Iteration  250, KL divergence 0.9448, 50 iterations in 0.2809 sec
   --> Time elapsed: 1.41 seconds
===> Running optimization with exaggeration=1.00, lr=200.00 for 750 iterations...
Iteration   50, KL divergence 0.0109, 50 iterations in 0.3351 sec
Iteration  100, KL divergence 0.0109, 50 iterations in 0.3248 sec
Iteration  150, KL divergence 0.0109, 50 iterations in 0.3070 sec
Iteration  200, KL divergence 0.0109, 50 iterations in 0.3073 sec
Iteration 

In [7]:
from sklearn.datasets import make_spd_matrix
D = make_spd_matrix(X.shape[1])
N = make_spd_matrix(X.shape[0])

f = partial(KL_divergence_dy, X_unflattener=X_unflattener, Y_unflattener=Y_unflattener)
H = jax.jacrev(f, argnums=1)(X_flat, Y_flat)
H_pinv = np.linalg.pinv(H + 1e-3*np.eye(len(H)), hermitian=True)

f = partial(KL_divergence_dy, Y_flat=Y_flat, X_unflattener=X_unflattener, Y_unflattener=Y_unflattener)
# jvp
_, jvp_fun_lin = jax.linearize(f, X_flat)
# vjp
_, vjp_fun = vjp(f, X_flat)

time_start = time.time()
compute_cov_fun = lambda i: compute_cov_inner(vjp_fun=vjp_fun, jvp_fun_lin=jvp_fun_lin, 
                                        H_pinv_i=i, D=D, N=N, d=D.shape[0], n=N.shape[0], H_pinv=H_pinv)
final_cov = vmap(compute_cov_fun)(H_pinv)
time_end = time.time()
print('time: ', time_end - time_start)

time:  1.9144871234893799


In [8]:
#print('KL divergence: ', KL_divergence(X_flat, Y_flat, X_unflattener, Y_unflattener))
#print('derivative of KL divergence w.r.t. Y: ', KL_divergence_dy(X_flat, Y_flat, X_unflattener, Y_unflattener))
#print('Hessian', H)
print('final covariance', final_cov)

final covariance [[ 6.22437801e-04  4.94177686e-04 -7.45397876e-04 ...  3.73561234e-05
   1.62843266e-04  1.53749439e-04]
 [ 4.94178385e-04  8.50181037e-04  7.12622132e-05 ... -3.85209860e-05
  -5.34712162e-05 -8.52075245e-05]
 [-7.45398167e-04  7.12624824e-05  5.09006064e-03 ... -3.36255820e-04
  -1.00966287e-03 -1.06366130e-03]
 ...
 [ 3.73560215e-05 -3.85210587e-05 -3.36255529e-04 ...  2.10113125e-04
   7.44308418e-05  1.09948145e-04]
 [ 1.62843324e-04 -5.34712672e-05 -1.00966287e-03 ...  7.44307981e-05
   3.46692163e-04  3.32814030e-04]
 [ 1.53749468e-04 -8.52074591e-05 -1.06366142e-03 ...  1.09948174e-04
   3.32814117e-04  4.45897196e-04]]


### Old 

In [9]:
def compute_dy_dx(f, X, Y):
  H = jax.hessian(f, argnums=1)(X_flat, Y_flat)
  #print('Hessian: ', H)
  H_pinv = np.linalg.pinv(H + 1e-3*np.eye(len(H)), hermitian=True)
  J_X_Y = jacrev(jacfwd(f, argnums=1), argnums=0)(X_flat, Y_flat)
  return np.dot(-H_pinv, J_X_Y)

f = partial(KL_divergence, X_unflattener=X_unflattener, Y_unflattener=Y_unflattener)
dy_dx = compute_dy_dx(f, X_flat, Y_flat)
final = np.dot(np.dot(dy_dx, np.kron(D, N)), dy_dx.T)
print('final cov: ', final)